In [1]:
import pandas as pd
import re
from symspellpy import SymSpell, Verbosity
import pkg_resources

df = pd.read_csv('data/patient_notes.csv', usecols=[2], encoding='utf-8')

print(df[:10])

                                          pn_history
0  17-year-old male, has come to the student heal...
1  17 yo male with recurrent palpitations for the...
2  Dillon Cleveland is a 17 y.o. male patient wit...
3  a 17 yo m c/o palpitation started 3 mos ago; \...
4  17yo male with no pmh here for evaluation of p...
5  17 yo m, presenting with palpitations/increase...
6  17 yo male presents with 3-4 month history of ...
7  17 yo male presents with a 2-3 month history o...
8  HPI: 17 YO complains of intermittent heart pal...
9  Mr. Cleveland is a 17 yo M who presents to cli...


In [2]:
# print(df['pn_history'])
print(df[:20])

                                           pn_history
0   17-year-old male, has come to the student heal...
1   17 yo male with recurrent palpitations for the...
2   Dillon Cleveland is a 17 y.o. male patient wit...
3   a 17 yo m c/o palpitation started 3 mos ago; \...
4   17yo male with no pmh here for evaluation of p...
5   17 yo m, presenting with palpitations/increase...
6   17 yo male presents with 3-4 month history of ...
7   17 yo male presents with a 2-3 month history o...
8   HPI: 17 YO complains of intermittent heart pal...
9   Mr. Cleveland is a 17 yo M who presents to cli...
10  Mr. Cleveland is a 17 year old male with no si...
11  Dillon Cleveland is a 17 year old male with no...
12  Patient is a 17 yo male presenting to student ...
13  17 yr old boy complaining of heart pounding fo...
14  17 year old male with no significant PMH. Comp...
15  Mr. Cleveland is a 17 yo male who presents tod...
16  HPI: 17yo M presents with palpitations. Patien...
17  17 yo male presents with

In [3]:
df['pn_history'] = df['pn_history'].apply(lambda x: x.lower())

In [4]:
df['pn_history'] = df['pn_history'].apply(lambda x: re.sub(r'[!@#$%^&*()_+=\-~`;:\'",.<>/?|\\]', '', x))

In [5]:
df['pn_history'] = df['pn_history'].apply(lambda x: re.sub(r'(\d{1,2})/(\d{1,2})/(\d{4})', r'\2-\1-\3', x))

In [6]:
import contractions

df['pn_history'] = df['pn_history'].apply(lambda x: contractions.fix(x))

In [7]:
import nltk
from nltk.corpus import stopwords
import pandas as pd

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
additional_stopwords = {'is', 'of', 'an'}

stop_words.update(additional_stopwords)
df['stopwords_removed'] = df['pn_history'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df['pn_history'][0]

'17yearold male has come to the student health clinic complaining of heart pounding mr clevelands mother has given verbal consent for a history physical examination and treatment\r\nbegan 23 months agosuddenintermittent for 2 dayslasting 34 minworseningnonallevaggrav\r\nassociated with dispnea on exersion and reststressed out about school\r\nreports fe feels like his heart is jumping out of his chest\r\nrosdenies chest paindyaphoresiswt losschillsfevernauseavomitingpedal edeam\r\npmhnonmeds aderol from a friendnkda\r\nfhfather had mi recentlymother has thyroid dz\r\nshnonsmokermariguana 56 months ago3 beers on the weekend basketball at school\r\nshno std'

In [9]:

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
sym_spell.load_dictionary( dictionary_path,term_index=0, count_index=1)

True

In [ ]:
def correct_spelling(text):
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
    return suggestions[0].term if suggestions else text

df['corrected_notes'] = df['stopwords_removed'].apply(correct_spelling)

In [ ]:
df.to_csv('processed_notes.csv', index=False)

In [9]:
processed = pd.read_csv('data/processed_notes.csv')

In [10]:
!pip install wordninja

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/541.6 kB ? eta -:--:--
      --------------------------------------- 10.2/541.6 kB ? eta -:--:--
     -- ---------------------------------- 41.0/541.6 kB 330.3 kB/s eta 0:00:02
     -------- --------------------------- 122.9/541.6 kB 901.1 kB/s eta 0:00:01
     -------------------- ----------------- 286.7/541.6 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 541.6/541.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541535 sha256=f2ff247c2d2d4257daf19a22cc76fc2ba7f24d73f35543046272449512c4b3aa
  Stored in directory: c:\users\harry\appdata\local\pip\cache\wheels\e6\66\9c\712044a983337f5d44f90abcd244bd4b8ad28ee64750404b50
Successfully built wordninja


In [13]:
wordninja.split('agosuddenintermittent')

['ago', 'sudden', 'intermittent']